In [30]:
import csv
import time
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

In [31]:
df = pd.read_csv("weekly_data_for_visualization.csv")
ids = df["Q317"].unique()
ids=ids[:-2] #only keeping the numbers of ids
ids

array(['117639', '144932', '158455', '186277', '294592', '312317',
       '313449', '325268', '351278', '441255', '446875', '453897',
       '481113', '483491', '483975', '494149', '495579', '547832',
       '575389', '738648', '752457', '773684', '855371', '865112',
       '873115', '918349', '931499', '953571'], dtype=object)

In [32]:
# create a list of dates from mili sec
def changeDateType(dates):
    date_final = [i.split(" ")[0] for i in dates]
    converted_dates = []
    for i in range(0,len(date_final)):  
        date_object = datetime.strptime(date_final[i], '%m/%d/%Y')
        converted_dates.append(date_object)
    return converted_dates

In [33]:
def changeYYMMDD(date):
    index = date.index("-")
    date = date[index-4:index+6]
    date = date.replace("-","/")
    date_object = datetime.strptime(date, '%Y/%m/%d')
    return date_object


In [34]:
def addBaseLine(idf,surveyKeep):
    df = pd.read_csv("sassema baseline 3.16.2023.csv")

    if idf == "773684":
        idf = "773"
    
    baselineDate = str(df.loc[df["ID"]==idf, 'StartDate'])

    # index = baselineDate.index("-")
    # baselineDate = baselineDate[index-4:index+6]
    # baselineDate = baselineDate.replace("-","/")
    # date_object = datetime.strptime(baselineDate, '%Y/%m/%d')
    date_object = changeYYMMDD(baselineDate)

    surveyKeep.insert(0,date_object)



In [48]:
dayDict={}
sensingData = pd.read_csv("Activity_joined.csv")
for id in ids:
    # dates 
    surveydates = df.loc[df["Q317"]==id, 'StartDate']     
    # add the survey dates of a user id to a list
    surveyKeep = changeDateType(surveydates)

    addBaseLine(id,surveyKeep)
    
    # create a list of surveys that we keep for each user 
    
    i = 1
    # discard current survey if the time interval between previous and current survey is not 7 
    while i < len(surveyKeep):

        numDays = surveyKeep[i] - surveyKeep[i-1]
        strDays = str(numDays)

        difference = strDays.split(" ")[0]
        difference = int(difference)

        if difference < 7:
            surveyKeep.pop(i) 

        else: 
            i = i + 1 


    #create a list of sensing data
    print(sensingData)
    sensingDates = sensingData.loc[sensingData["Userid"]==id, ' senseStartTime']
    sensingKeep = []
    print("sensing dates outside", sensingDates)
    for i in range(0,len(sensingDates)):
        print("sensing dates", sensingDates[i])
        date_object = changeYYMMDD(sensingDates[i])
        sensingKeep.append(date_object)

    # sensingKeep = changeYYMMDD(sensingDates)


    # sensingDates = changeDateType


    # for each survey look back at sensing data 7 days before and see how many days of data we have 
            
#         index = str(numDays)
#         key = index.split(" ")[0]
#         key = int(key)
#         if key not in dayDict:
#             dayDict[key] = 1
#         else:
#             dayDict[key] += 1
        
# # print(surveyKeep)
# myKeys = list(dayDict.keys())
# myKeys.sort()
# dayDict = {i: dayDict[i] for i in myKeys}
# print(dayDict)

                  deviceid   senseStartTimeMillis   sensorType  \
0       '0b482745b4fced28'          1649448366616   'Activity'   
1       '0b482745b4fced28'          1649449588320   'Activity'   
2       '0b482745b4fced28'          1649449940220   'Activity'   
3       '0b482745b4fced28'          1649449941645   'Activity'   
4       '0b482745b4fced28'          1649456847630   'Activity'   
...                    ...                    ...          ...   
123547  '071a04e73a407b0d'          1673630764524   'Activity'   
123548  '071a04e73a407b0d'          1673630946435   'Activity'   
123549  '071a04e73a407b0d'          1673631127452   'Activity'   
123550  '071a04e73a407b0d'          1673631308447   'Activity'   
123551  '071a04e73a407b0d'          1673631506804   'Activity'   

                senseStartTime   version     os   Confidence    Activity  \
0        '2022-04-08T20:06:06'   '2.3.1'   '12'          100     'Still'   
1        '2022-04-08T20:26:28'   '2.3.1'   '12'        